In [1]:
import time
import copy
import random
import astar
import environment
import model
import numpy as np
from tabulate import tabulate
import pprint
from IPython.core.display import display, HTML
from collections import deque
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam, RMSprop
display(HTML("<style>.container { width: 100% !important; }</style>"))

Using TensorFlow backend.


In [2]:
patrol_env = environment.env()
temp=astar.maze
dqn = model.Dqn_network()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 64)                117824    
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 260       
Total params: 122,244
Trainable params: 122,244
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 64)                117824    
_________________________________________________________________
dense_5 (Dense)              (None, 64)                4160      
__________________________

In [3]:
#reward_adjust constant
minus_reward=[]
init=[0,3]
for i in range(7):
    result=[]
    goal=temp.goal_state[i]
    astar_Solver = astar.Aster_Solver(temp, init, goal , display=False)
    result=result+astar_Solver.solve_maze()
    reward = temp.game_original[temp.goal_state[i][0]][temp.goal_state[i][1]]
    reward= np.round(len(result)/reward,2)
    minus_reward.append(reward)
patrol_env.minus_reward=minus_reward

Sucess!
Sucess!
Sucess!
Sucess!
Sucess!
Sucess!
Sucess!


In [ ]:
#adjust_reward
episode = 500
dqn.epsilon=1.0
for e in range(episode):
    patrol_env.reset()
    step=0
    state=[0,0]
    tp=0
    score=0
    ret=4        
    while(True):
        step=step+1
        action=dqn.choose_action(state)
        next_state,reward,done = patrol_env.act(action)
        dqn.remember_memory(state, action, reward, next_state, done)
        #print(state[0],maze.states[state[1]][1],maze.states[state[1]][2],next_state[0],maze.states[next_state[1]][1],maze.states[next_state[1]][2],reward,done)
        #print(maze.curr_state[0],maze.states[maze.curr_state[1]][1],maze.states[maze.curr_state[1]][2])
        state=next_state
        if reward>=1:
            score=score+reward
            tp=tp+1
            if state[0]!=3:
                state[0]=state[0]+1
            print(reward,done,step)
        if step>20000:
            print(state[0],patrol_env.states[state[1]][1],patrol_env.states[state[1]][2],next_state[0],patrol_env.states[next_state[1]][1],patrol_env.states[next_state[1]][2],reward,done)
            #retry=int(score//ret)
            dqn.replay_experience()
            print("fail")
            break

            #dqn.update_target_model() 
        if done:
            patrol_env.adjust_reward()
            #retry = int(score//4)
            dqn.replay_experience()  
            if dqn.epsilon > dqn.e_min:
                dqn.epsilon *= dqn.e_decay
            print("episode: {}/{},  e: {:.2} \t @ {}"
                        .format(e, episode, dqn.epsilon,step))
            #dqn.replay_experience()  
            dqn.update_target_model() 
            break

45.0 False 669
7.0 False 1554
22.0 False 2155
67.0 True 2343
episode: 0/500,  e: 1.0 	 @ 2343
45.0 False 414
67.0 False 1426
33.0 False 2154
229.0 True 4393
episode: 1/500,  e: 1.0 	 @ 4393
22.0 False 723
67.0 False 1587
229.0 False 5705
7.0 True 7602
episode: 2/500,  e: 1.0 	 @ 7602
4.43 False 1239
20.45 False 1809
66.03 False 4757
228.79 True 5464
episode: 3/500,  e: 1.0 	 @ 5464
4.43 False 543
20.45 False 1150
66.03 False 2145
228.79 True 4330
episode: 4/500,  e: 1.0 	 @ 4330
44.13 False 590
228.36999999999998 False 3596
66.03 False 13254
17.349999999999998 True 16003
episode: 5/500,  e: 1.0 	 @ 16003
44.13 False 148
228.36999999999998 False 4404
66.03 False 5622
17.349999999999998 True 12020
episode: 6/500,  e: 1.0 	 @ 12020
44.13 False 719
17.349999999999998 False 2311
66.03 False 3365
97.95 True 4957
episode: 7/500,  e: 1.0 	 @ 4957
44.13 False 403
228.15999999999997 False 4083
12.699999999999996 False 5180
61.18000000000001 True 9996
episode: 8/500,  e: 1.0 	 @ 9996
12.699999999

46.630000000000024 False 2592
83.92000000000002 False 3440
224.58999999999983 True 15245
episode: 65/500,  e: 0.99 	 @ 15245
41.81000000000001 False 297
46.630000000000024 False 8759
83.92000000000002 False 11076
3 43 0 3 43 0 0 False
fail
81.48000000000002 False 1506
46.630000000000024 False 1831
36.010000000000026 False 6722
3 45 17 3 45 17 0 False
fail
41.81000000000001 False 914
46.630000000000024 False 3368
223.5399999999998 False 8218
3 10 11 3 10 11 0 False
fail
41.81000000000001 False 251
83.31000000000002 False 2998
49.54000000000002 False 9800
224.58999999999983 True 10570
episode: 69/500,  e: 0.99 	 @ 10570
34.99000000000004 False 5166
223.32999999999979 False 9860
2 17 2 2 17 2 0 False
fail
41.81000000000001 False 700
83.31000000000002 False 3136
49.54000000000002 False 6680
224.58999999999983 True 16258
episode: 71/500,  e: 0.99 	 @ 16258
41.81000000000001 False 518
44.690000000000026 False 1472
223.11999999999978 False 1811
81.48000000000002 True 14155
episode: 72/500,  e

28.47000000000005 False 5484
69.89000000000003 False 8631
3 26 21 3 26 21 -0.01 False
fail
40.360000000000014 False 3415
219.75999999999965 False 4415
69.89000000000003 False 5856
12.680000000000057 True 19133
episode: 131/500,  e: 0.98 	 @ 19133
40.360000000000014 False 221
71.72000000000003 False 2324
34.99000000000004 False 3634
3 39 17 3 39 17 0 False
fail
40.360000000000014 False 253
71.72000000000003 False 3144
34.99000000000004 False 18228
3 35 18 3 35 18 0 False
fail
40.360000000000014 False 1588
219.75999999999965 False 4309
34.99000000000004 False 11485
3 41 12 3 41 12 -0.01 False
fail
65.01000000000003 False 3860
28.18000000000005 False 6512
219.33999999999963 False 10212
12.680000000000057 True 17604
episode: 135/500,  e: 0.98 	 @ 17604
40.070000000000014 False 2104
25.29000000000005 False 5096
69.28000000000003 False 8136
3 25 0 3 25 0 0 False
fail
40.070000000000014 False 2413
219.54999999999964 False 9110
34.02000000000004 False 11209
3 11 43 3 11 43 0 False
fail
40.0700

In [ ]:
#dqn or qlearning(faster)
Q_model=model.Q_learning()
Q_model.env = patrol_env
Q_model.learn()

In [ ]:
Q_model.learn_back()

In [ ]:
print(Q_model.path)